In [ ]:
!pip install pydub
!pip install ffmpeg

from scipy import signal
import librosa
import IPython.display as ipd
from scipy.io import wavfile
import numpy as np
import os
import wave
import pylab
import sys
import math
import pandas as pd
import csv
import matplotlib.pyplot as plt

In [ ]:
#Connects to your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Path for CSV where the clipped audio information will be put
audioCSVPath = '/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/AudioTrainingDataSet/AudioTrainingSheet.csv'

In [ ]:
audiocsv = pd.read_csv(audioCSVPath)

In [ ]:
audiocsv['target'].unique()

array([ 0, 14, 36, 19, 30, 34,  9, 22, 48, 41, 47, 31, 17, 45,  8, 15, 46,
       37, 32, 16, 25,  4,  3, 27, 43, 12, 40, 29, 10,  7, 26,  6, 44, 23,
       20, 49, 24, 39, 28, 18,  2, 35, 38, 21,  1, 11, 42,  5, 33, 13, 51])

In [ ]:
#Locations of Audio Data in a google drive
#Edit based on your paths

#Raw training audio locations
RAW_AUDIO_DIRS = ['/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/RawAcadiaAudio/Cadillac',
                  '/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/RawAcadiaAudio/Pinehill',
                  '/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/RawAcadiaAudio/Bernard',
                  '/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/RawAcadiaAudio/Gilmore',
                  '/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/RawAcadiaAudio/Northeast']


#Location for where the clipped audio will be stored
CLIP_AUDIO_DIR = '/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/AudioTrainingDataSet/audiofiles/'


In [ ]:
#If you create a new csv and refill it with data be sure to uncomment and run this cell to create a new line in the CSV file otherwise it will creat formatting issues
#with open(audioCSVPath, 'a') as f:
#   f.write('\n')

In [ ]:

#helper function for finding location from file name
def LocationHelper(filename):
  if (filename[0] == 'C'):
    return 'cadillac', 6
  elif (filename[0] == 'P'):
    return 'pinehill', 7
  elif (filename[0] == 'B'):
    return 'bernard', 8
  elif (filename[0] == 'G'):
    return 'gilmore', 9
  elif (filename[0] == 'N'):
    return 'northeast', 10

#helper for finding the category name from the target
def CategoryHelper(target):
  cat = audiocsv['category'].loc[audiocsv['target'] == target]
  return cat.unique()[0]

#helper for finding target from file name
def TargetHelper(filename):
  temp = filename.split('-')
  return temp[1], int(temp[2])

In [ ]:
#helper for splitting audio into chunks
def SplitAudio(filename, samples, sample_chunks, sample_rate):

  df = pd.DataFrame(columns=['filename','fold', 'target', 'category', 'src_file', 'take', 'location'])

  location, fold = LocationHelper(filename)
  src, target = TargetHelper(filename)
  category = CategoryHelper(target)
  take = 0

  for i in range(math.floor(samples.size / sample_chunks)):
    frame_a = i  * sample_chunks
    frame_b = (i+1) * sample_chunks
    take = i
    #print(take)
    filename = str(fold) + '-' + src + '-' + chr(take + 65) + '-' + str(target)
    if not os.path.exists(CLIP_AUDIO_DIR + location + '/' + filename + '.wav'):
      print(chr(take + 65))
      wavfile.write(CLIP_AUDIO_DIR + location + '/' + filename + '.wav', sample_rate, samples[frame_a:frame_b])
      df.loc[take] = [filename, fold, target, category, src, chr(take + 65), location]


  df.to_csv(audioCSVPath, mode='a',header=False, index=False)


In [ ]:

#samples, sample_rate = librosa.load(RAW_AUDIO_DIRS[0] + '/' + 'C-0622093957-47.wav')
#sample_chunks = 5 * sample_rate

#SplitAudio('C-0622093957-47', samples, sample_chunks, sample_rate)

In [ ]:
#Processes an individual locations training data
def ProcessAudioDir(fileLoc):
  #change based on the desired size of clipped training data in seconds
  time = 5

  for filename in os.listdir(fileLoc):
      if "wav" in filename:
        samples, sample_rate = librosa.load(fileLoc + '/' + filename)
        sample_chunks = time * sample_rate

        filenamenowav = filename.split('.')[0]
        SplitAudio(filenamenowav, samples, sample_chunks, sample_rate)



In [ ]:
#ProcessAudioDir('/content/drive/MyDrive/Datasets/SoundRecog/Natural_Sound_Dataset/RawAcadiaAudio/Pinehill')

In [ ]:
#function to process all raw acadia training data
def ProcessAllAcadiaData(RAW_AUDIO_DIRS):
  for dir in RAW_AUDIO_DIRS:
    print(dir)
    ProcessAudioDir(dir)
  print('Done Processing')

In [ ]:
#Run this to process all raw audio files into the training data
ProcessAllAcadiaData(RAW_AUDIO_DIRS)